In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

In [ ]:
!unzip /gdrive/MyDrive/CelebA_dataset/celeba-dataset.zip -d /content/


In [ ]:
import pandas as pd
from keras.models import Model 
from keras.layers import Dropout, Dense, BatchNormalization
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

In [ ]:
num_features = 4



complete_dataset_df = pd.read_csv ('/content/celeba-dataset/list_attr_celeba.csv', usecols=[0,2,3,4,5])

ev_part = pd.read_csv ('/content/celeba-dataset/list_eval_partition.csv')



def build_model(num_features):
  base = MobileNetV2(input_shape=(224, 224, 3),
                     weights=None,
                     include_top=False,
                     pooling='avg')  
  
  # model top
  x = base.output
  x = Dense(100, activation='relu')(x)
  x = BatchNormalization()(x)
  x = Dropout(0.3)(x)
  top = Dense(num_features, activation='sigmoid')(x)
  
  return Model(inputs=base.input, outputs=top)


model = build_model(num_features=num_features)
model.summary()


In [ ]:
training_set = complete_dataset_df.drop(ev_part[(ev_part['partition']!=0)].index)
validation_set = complete_dataset_df.drop(ev_part[(ev_part['partition']!=1)].index)
test_set = complete_dataset_df.drop(ev_part[(ev_part['partition']!=2)].index)

#training_set.set_index('image_id', inplace=True)
#validation_set.set_index('image_id', inplace=True)

features_name= list(complete_dataset_df.columns)[1:]
print(training_set)




In [ ]:


train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)



train_generator = train_datagen.flow_from_dataframe(
    dataframe=training_set,
    directory='/content/celeba-dataset/img_align_celeba',
    x_col='image_id',
    y_col=features_name,
    target_size=(224, 224),
    batch_size=100,
    class_mode='raw'
)

valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=validation_set,
    directory='/content/celeba-dataset/img_align_celeba',
    x_col='image_id',
    y_col=features_name,
    target_size=(224, 224),
    batch_size=100,
    class_mode='raw'
)




In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adadelta',
              metrics='binary_accuracy')

In [ ]:

model_path = f"/content/weights-FC{num_features}-MobileNetV2" + "{val_binary_accuracy:.2f}.hdf5"

checkpoint = ModelCheckpoint(
    model_path,
    monitor='val_binary_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1)


history = model.fit(
    train_generator,
    epochs=3,
    steps_per_epoch=len(train_generator),
    validation_data=valid_generator,
    validation_steps=len(valid_generator),
    max_queue_size=1,
    shuffle=True,
    callbacks=[checkpoint],
    verbose=1)

In [ ]:
import matplotlib.pyplot as plt


def plot_model_history(history):

  plt.plot(history.history['binary_accuracy'])
  plt.plot(history.history['val_binary_accuracy'])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validation'], loc='upper left')
  plt.show()

  
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validation'], loc='upper left')
  plt.show()

  
plot_model_history(history)